Sorry, this is really rough, currently this has some initial EDA I did on the various subreddits I'm thinking of scraping as well as the backbone of a potential scraping function.

Some things I need to research and/or ask about

1) I'm currently thinking about saving this as a list of nested dictionaries. Need to determine if that's the best idea or not

2) I'm worried I'm hitting reddit too much (1,000 comments per 5 subs) * (up to 1,000 posts per unique user). Researched that a bit and I think praw has a built in 2 sec delay so things are probably okay but probably should research a bit more and break up my scrapes

3) Currently, my code gives equal weight to a comment poster but that's probably not ideal, especially since people can go onto a subreddit and comment even though they're not a supporter. Would ideally like to do something where I account for this.

4) My code currently doesn't have a way to append or add more data, need to consider how I might go about doing that


In [1]:
import praw
import json
import time
import pandas as pd
import sys
import numpy as np

In [2]:
sys.version

'3.7.3 (default, Mar 27 2019, 16:54:48) \n[Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
praw.__version__

'6.3.1'

In [2]:
json_file = open('Reddit_creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [3]:
reddit = praw.Reddit(user_agent=json_data['user_agent'],
                     client_id=json_data['client_id'], client_secret=json_data['client_secret'])

In [4]:
yang_sub = reddit.subreddit('YangForPresidentHQ')

In [5]:
pete_sub = reddit.subreddit('Pete_Buttigieg')

In [6]:
sanders_sub = reddit.subreddit('SandersForPresident/')

In [7]:
harris_sub = reddit.subreddit('Kamala')

In [8]:
biden_sub = reddit.subreddit('JoeBiden')

In [9]:
subs = [yang_sub,pete_sub,sanders_sub,harris_sub,biden_sub]

In [10]:
users = []

In [11]:
start = time.time()
for sub in subs:
    for comment in sub.comments(limit=1000):
        users.append(comment.author)
        
print(len(users))
users = list(set(users))
print(len(users))
end = time.time()
print(end-start)

4679
1683
48.893178939819336


In [37]:
# nums = np.random.randint(1,1733,100)

In [39]:
# short_list = []
# for x in nums:
#     short_list.append(users[x])

In [1]:
# short_list

In [ ]:
# #code obtained from https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# # load additional module
# import pickle

# # define a list of places
# with open('users.data', 'wb') as filehandle:
#     # store the data as binary data stream
#     pickle.dump(set(users), filehandle)

In [ ]:
#code obtained from https://stackabuse.com/reading-and-writing-lists-to-a-file-in-python/
# with open('users.data', 'rb') as filehandle:
#     # read the data as binary data stream
#     test_set = pickle.load(filehandle)

In [ ]:
# small_list2 = list(set(users))[100:200]

In [ ]:
# len(small_list)

In [ ]:
#First full scrape started around 11AM on 8/6/19
#Second full scrape started at 1:13 PM on 8/6/19

In [12]:
start = time.time()
text = []
author = []
subreddit = []
postid = []
counter = 0
for user in users:
    if counter % 10 == 0:
        prog = time.time()
        print(counter)
        print((prog-start)/60)
    try:  #handle cases where redditor is deleted or something
        for comment in user.comments.new(limit=1000):
            text.append(comment.body)
            author.append(comment.author)
            postid.append(comment.id)
            subreddit.append(comment.subreddit.display_name)
    except:
        pass
    counter += 1
end = time.time()
print(end-start)

0
3.0835469563802084e-06
10
1.8631019194920857
20
3.6755609194437664
30
5.407702700297038
40
6.430276433626811
50
9.48271341721217
60
10.603218233585357
70
15.615761148929597
80
16.557903798421226
90
17.62286973396937
100
18.73901447057724
110
21.104294951756795
120
91.47355191707611
130
92.0931421995163
140
93.2501658320427
150
94.24871366818746
160
94.78590141534805
170
95.82262081702551
180
96.69580888350805
190
97.5403844833374
200
98.38021794954936
210
99.73189623355866
220
100.61249213616053
230
101.07335486809413
240
102.11790694793065
250
103.01888636747996
260
103.91995456616084
270
105.09508270025253
280
105.85599674781163
290
106.79565949837367
300
107.84694071610768
310
108.77555008331935
320
109.78739306926727
330
110.96982630093892
340
111.47191054821015
350
112.42461916605632
360
113.20736796458563
370
114.07782853047053
380
115.13772791624069
390
115.9722027182579
400
117.5712100982666
410
118.5167038679123
420
119.12836628357569
430
120.24419938325882
440
134.732057034

In [13]:
df = pd.DataFrame([text,author,subreddit,postid],\
                        index=['Comment Text','Author','Subreddit','Post ID']).T

In [14]:
df.shape

(1024524, 4)

In [15]:
df.to_csv('full_scrape_1.csv',index=False)

In [ ]:
df.iloc[1,0]

In [ ]:

for user in set(users):
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if dict_sub.get(comment.subreddit.display_name,0) == 0:
            dict_sub[comment.subreddit.display_name] = 1
        else:
            dict_sub[comment.subreddit.display_name] += 1
    yang_dict[user] = dict_sub
#     print(f'finished {user}')
    time.sleep(5)

In [ ]:

yang_users = []
for comment in yang_sub.comments(limit=1000):
    yang_users.append(comment.author)
print(len(yang_users))
print(len(set(yang_users)))


In [ ]:
setyang = set(yang_users)

In [ ]:
harris_users = []
for comment in harris_sub.comments(limit=1000):
    harris_users.append(comment.author)
print(len(harris_users))
print(len(set(harris_users)))

In [ ]:
setharris = set(harris_users)

In [ ]:
biden_users = []
for comment in biden_sub.comments(limit=1000):
    biden_users.append(comment.author)
print(len(biden_users))
print(len(set(biden_users)))

In [ ]:
setbiden = set(biden_users)

In [ ]:
sanders_users = []
for comment in sanders_sub.comments(limit=1000):
    sanders_users.append(comment.author)
print(len(sanders_users))
print(len(set(sanders_users)))

In [ ]:
setsanders = set(sanders_users)

In [ ]:
pete_users = []
for comment in pete_sub.comments(limit=1000):
    pete_users.append(comment.author)
print(len(pete_users))
print(len(set(pete_users)))

In [ ]:
setpet = set(pete_users)

In [ ]:
setnotyang = set(pete_users+sanders_users+biden_users+harris_users)

In [ ]:
len(setnotyang)

In [ ]:
count = 0
for user in setyang:
    if user in setnotyang:
        print(user)
        if user in setbiden:
            print('+biden')
        if user in setharris:
            print('+harris')
        if user in setsanders:
            print('+sanders')
        if user in setpet:
            print('+pete')
        count += 1

In [ ]:
print(count)

Okay, feeling good, out of the 505 most recent yang posters, only 4 posted to one of these other subs, checking other subs

In [ ]:

for user in setbiden:
    if user in setharris:
        print('+harris')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')


In [ ]:

for user in setharris:
    if user in setbiden:
        print('+biden')
    if user in setsanders:
        print('+sanders')
    if user in setpet:
        print('+pete')

In [ ]:
count = 0
for user in setpet:
    if user in setsanders:
        count += 1
        
print(count)

In [ ]:
len(setpet)

In [ ]:
len(setsanders)

In [ ]:
len(set(list(setpet)+list(setsanders)))

In [ ]:
171+277

In [ ]:
#backbone scraping code, idea would be to run this for every user in each subreddits set of users
yang_dict = {}
for user in yang_users[22:25]:
    dict_sub = {}
    for comment in user.comments.new(limit=1000):
        if dict_sub.get(comment.subreddit.display_name,0) == 0:
            dict_sub[comment.subreddit.display_name] = 1
        else:
            dict_sub[comment.subreddit.display_name] += 1
    yang_dict[user] = dict_sub
#     print(f'finished {user}')
    time.sleep(5)

In [ ]:
yang_dict